In [1]:
import warnings

warnings.filterwarnings('ignore')

In [2]:
import torch
from torch_geometric.data import Data
import networkx as nx
import networkit as nk
import numpy as np
import polars as pl

In [36]:
import torch.nn as nn
import torch

In [3]:
import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import igraph as ig
from torch_geometric.loader import NeighborLoader
import tqdm
from multiprocessing import Pool

In [4]:
import os
import sys

sys.path.append('../')
sys.path.append('/Users/phamminhlong/Desktop/paper')

In [5]:
ROOT = os.getcwd()
WORK_DIR = os.path.dirname(os.path.dirname(ROOT))

## ***Test code***

In [6]:
FIRST_FEAT_NAME = {
        'column_1': 'transid',
        'column_2': 'time_steps',
}

In [7]:
df_classes = pl.read_csv(os.path.join(WORK_DIR, 'data/elliptic_bitcoin_dataset/elliptic_txs_classes.csv'), new_columns=['transid', 'class'])
df_edgelist = pl.read_csv(os.path.join(WORK_DIR, 'data/elliptic_bitcoin_dataset/elliptic_txs_edgelist.csv'), new_columns=['current_transid', 'next_transid'])
df_features = pl.read_csv(os.path.join(WORK_DIR, 'data/elliptic_bitcoin_dataset/elliptic_txs_features.csv'), has_header=False)

In [8]:
from src.service.data_loader import EllipticLoader
from src.service.graph_model import GAT

In [9]:
e = EllipticLoader(
    path_classes='data/elliptic_bitcoin_dataset/elliptic_txs_classes.csv',
    path_edgelist='data/elliptic_bitcoin_dataset/elliptic_txs_edgelist.csv',
    path_features='data/elliptic_bitcoin_dataset/elliptic_txs_features.csv'   
)

In [37]:
hidden_dim=64
embedding_dim=128
n_layers=3
n_features=166
output_dim=1
dropout_rate=0.6
heads=5
batch_size=128
lr=1e-5

In [16]:
gat = GAT(
    num_features=n_features,
    hidden_dim=hidden_dim,
    embedding_dim=embedding_dim,
    output_dim=output_dim,
    n_layers=n_layers,
    heads=heads,
    dropout_rate=dropout_rate
)

In [17]:
data = e.load()

2024-06-23 12:17:46,733 - src.utils.timer - INFO - DONE PROCESS PHASE ELLIPTIC LOADER IN 0.4786s


INFO:src.utils.timer:DONE PROCESS PHASE ELLIPTIC LOADER IN 0.4786s


In [18]:
loader = NeighborLoader(
                data=data.get_network_torch(), 
                num_neighbors=[-1]*gat.n_layers, 
                input_nodes=data.train_mask, 
                batch_size=batch_size, 
                shuffle=True, 
                num_workers=Pool()._processes
            )

In [28]:
data.get_network_torch().x = data.get_network_torch().x[:, 1:94]

In [30]:
data.get_network_torch().x.shape

torch.Size([203769, 166])

In [41]:
gat.train()
optimizer = torch.optim.SGD(gat.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()
for i in loader:
    out, h = gat(i.x, i.edge_index)
    y_hat = out[:i.batch_size]
    y = i.y[:i.batch_size]
    loss = criterion(y_hat, y)
    loss.backward()
    print(loss)
    optimizer.step()

tensor(0.4920, grad_fn=<NllLossBackward0>)
tensor(0.4951, grad_fn=<NllLossBackward0>)
tensor(0.5210, grad_fn=<NllLossBackward0>)
tensor(0.5187, grad_fn=<NllLossBackward0>)
tensor(0.4915, grad_fn=<NllLossBackward0>)
tensor(0.4934, grad_fn=<NllLossBackward0>)
tensor(0.4916, grad_fn=<NllLossBackward0>)
tensor(0.5092, grad_fn=<NllLossBackward0>)
tensor(0.4992, grad_fn=<NllLossBackward0>)
tensor(0.5064, grad_fn=<NllLossBackward0>)
tensor(0.4523, grad_fn=<NllLossBackward0>)
tensor(0.4859, grad_fn=<NllLossBackward0>)
tensor(0.5270, grad_fn=<NllLossBackward0>)
tensor(0.4992, grad_fn=<NllLossBackward0>)
tensor(0.5381, grad_fn=<NllLossBackward0>)
tensor(0.4710, grad_fn=<NllLossBackward0>)
tensor(0.4527, grad_fn=<NllLossBackward0>)
tensor(0.4835, grad_fn=<NllLossBackward0>)
tensor(0.5016, grad_fn=<NllLossBackward0>)
tensor(0.5121, grad_fn=<NllLossBackward0>)
tensor(0.4895, grad_fn=<NllLossBackward0>)
tensor(0.5068, grad_fn=<NllLossBackward0>)
tensor(0.4950, grad_fn=<NllLossBackward0>)
tensor(0.53